In [1]:
import polars as pl
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, date2num
import time

#### Pull Data

In [ ]:
df = pl.read_csv(r"C:\Users\Denne\OneDrive\Documents\GitHub\FinancialStreamline2\BankStreamline\Data\BankDataProd.csv", ignore_errors=True)

df.limit(3)

date,category,description,cost,controllable,monthName,month,day,year,weekDay,cardType,quarter
str,str,str,f64,i64,str,i64,i64,i64,str,str,i64
"""2025-3-31""","""food""","""DC THE BLACK C…",54.61,1,"""Mar""",3,31,2025,"""Mon""","""Debit""",1
"""2025-3-31""","""amazon""","""DC eBay O*08-1…",125.74,1,"""Mar""",3,31,2025,"""Mon""","""Debit""",1
"""2025-3-28""","""wmt""","""POS 0328 0135 …",58.47,1,"""Mar""",3,28,2025,"""Fri""","""Debit""",1


#### Edit dataframe

In [ ]:
# nf = df.with_columns(category = pl.when(pl.col('description').str.contains('AIRBNB|EXPEDIA')).then(pl.lit('fun')) \
#         .otherwise(pl.col('category')))


nf = df.with_columns(category = pl.when(pl.col('category') == 'rent').then(pl.lit('mortgage')) \
        .otherwise(pl.col('category')))


nf = df.with_columns(category = pl.when(pl.col('category') == 'progressive').then(pl.lit('auto insurance')) \
        .otherwise(pl.col('category')))


# nf = df.with_columns(quarter = pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").dt.quarter())
                                    

# nf = nf.select(['date', 'category', 'description', 'cost', 'controllable', 'monthName', 'month', 'day', 'year', 'weekDay', 'cardType', 'quarter'])
# nf = nf.cast({'controllable':pl.Int16})


# nf['cost'].min()

# nf.filter(pl.col('controllable') == 'error')
# nf.filter((pl.col('category') == 'fun') & (pl.col('description').str.contains('EXPEDIA|AIRBNB')))
# nf.filter(pl.col('description').str.contains('DEPOSIT'))

nf

date,category,description,cost,controllable,monthName,month,day,year,weekDay,cardType,quarter
str,str,str,f64,i64,str,i64,i64,i64,str,str,i64
"""2025-3-31""","""food""","""DC THE BLACK C…",54.61,1,"""Mar""",3,31,2025,"""Mon""","""Debit""",1
"""2025-3-31""","""amazon""","""DC eBay O*08-1…",125.74,1,"""Mar""",3,31,2025,"""Mon""","""Debit""",1
"""2025-3-28""","""wmt""","""POS 0328 0135 …",58.47,1,"""Mar""",3,28,2025,"""Fri""","""Debit""",1
"""2025-3-27""","""wmt""","""POS 0327 0005 …",6.42,1,"""Mar""",3,27,2025,"""Thu""","""Debit""",1
"""2025-3-25""","""subscriptions""","""DC Spotify USA…",11.99,1,"""Mar""",3,25,2025,"""Tue""","""Debit""",1
"""2025-3-24""","""water""","""SHARE CHECK""",25.62,0,"""Mar""",3,24,2025,"""Mon""","""Debit""",1
"""2025-3-24""","""food""","""POS 0324 1312 …",1.75,1,"""Mar""",3,24,2025,"""Mon""","""Debit""",1
"""2025-3-24""","""internet""","""VIASAT, INC. …",137.66,0,"""Mar""",3,24,2025,"""Mon""","""Debit""",1
"""2025-3-24""","""wmt""","""POS 0324 0328 …",64.23,1,"""Mar""",3,24,2025,"""Mon""","""Debit""",1


#### Check for duplicates

In [ ]:
nf.filter(nf.is_duplicated())
# len(nf.filter(nf.is_duplicated()))

date,category,description,cost,controllable,monthName,month,day,year,weekDay,cardType,quarter
str,str,str,f64,i64,str,i64,i64,i64,str,str,i64
"""2025-3-3""","""misc""","""VENMO …",1.0,1,"""Mar""",3,3,2025,"""Mon""","""Debit""",1
"""2025-3-3""","""misc""","""VENMO …",1.0,1,"""Mar""",3,3,2025,"""Mon""","""Debit""",1


#### check nf length
- Mar: 784
- Apr: 853

In [9]:
len(nf)

853

#### Update file (use w/ caution)

In [ ]:
# path = r'C:\Users\Blake Dennett\Downloads\BankStreamline\Data\BankDataProd.csv'


# with open(path, mode="w") as f:
#    nf.write_csv(f, has_header=True)

#### Adhoc looking at data

In [57]:
# list(df.filter(pl.col('category') == 'unknown')['description'].unique())
list(nf['quarter'].unique())

# maybe car words: 'O.REILLY|O.REILLY|MUFFLEX MUFFLER|AUTOZONE|DC TAKE 5|KARL MALONE FORD', 

[1, 2, 3, 4]